# 可视化

使用wandb进行mask可视化, 需要以下：
1. image
2. pred_mask
3. gt_mask

In [ ]:
import wandb

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1073629662.py, line 6)

从数据集取出来的image和label都是在0，1之间，且image已经被transposed了，所以需要对其进行处理

In [ ]:
import torch



模型的输出是(batch_size, 2, H, W)

需要对模型的输出进行如下处理：
1. 

In [ ]:
def crf_process(model, image):
    import torch.nn.functional as F
    
    # Get model prediction
    logit = model(image)
    
    # Resize and get probabilities
    H, W = image.shape[-2:]
    logit = F.interpolate(logit, size=(H, W), mode="bilinear", align_corners=False)
    prob = F.softmax(logit, dim=1)[0].cpu().numpy()
    
    # Convert image for CRF
    img_np = (image[0].permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
    
    return prob, img_np


In [ ]:
def visualize_masks(image, pred_mask, gt_mask):
    """Visualize image and masks using wandb"""
    return wandb.Image(
        image,
        masks={
            "prediction": {"mask_data": pred_mask},
            "ground_truth": {"mask_data": gt_mask}
        }
    )
